# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
from IPython.display import Markdown
import os
from time import time
import json
from pprint import pprint

# import crewai   # Version: 0.30.11
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

from crewai_tools import (
    SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool,
    DirectoryReadTool, FileReadTool, BaseTool
)

from langchain_openai import ChatOpenAI

from pydantic import BaseModel

from api_key_store import ApiKeyStore
from helper import convert_to_filename

# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
s = ApiKeyStore()

os.environ["OPENAI_API_KEY"] = s.get_api_key(provider="OPENAI")  # os.environ["API_KEY_OPENAI"]
os.environ["SERPER_API_KEY"] = s.get_api_key(provider="SerpApi")  # os.environ["API_KEY_SERPAPI"]

model_name = "gpt-3.5-turbo"
os.environ["OPENAI_MODEL_NAME"] = model_name   # gpt-4 was default, throws rate-limit error 

## crewAI Tools

In [3]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [4]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time  to identify trends and predict market movements.",
    backstory="""
        "Specializing in financial markets, this agent "
          "uses statistical modeling and machine learning "
          "to provide crucial insights. With a knack for data, "
          "the Data Analyst Agent is the cornerstone for "
          "informing trading decisions."
    """.replace('"',''),
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [5]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based on insights from the Data Analyst Agent.",
    backstory="""
        Equipped with a deep understanding of financial markets and quantitative analysis, 
        this agent  devises and refines trading strategies. 
        It evaluates  the performance of different approaches to determine
        the most profitable and risk-averse options.
    """.replace('"',''),
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies  based on approved trading strategies.",
    backstory="""
        This agent specializes in analyzing the timing, price,  and logistical details of potential trades. 
        By evaluating  these factors, it provides well-founded suggestions for 
        when and how trades should be executed to maximize  efficiency and adherence to strategy.
    """.replace('"',''),
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks  associated with potential trading activities.",
    backstory="""
        Armed with a deep understanding of risk assessment models  and market dynamics, 
        this agent scrutinizes the potential  risks of proposed trades. 
        It offers a detailed analysis of  risk exposure and suggests safeguards to ensure that  
        trading activities align with the firm’s risk tolerance.
    """.replace('"',''),
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [8]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description="""
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    """.replace('"',''),
    expected_output= "Insights and alerts about significant market  opportunities or threats for {stock_selection}.",
    agent=data_analyst_agent,
)

In [9]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description="""
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    """.replace('"',''),
    expected_output="A set of potential trading strategies for {stock_selection} that align with the user's risk tolerance.",
    agent=trading_strategy_agent,
)


In [10]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description="""
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    """.replace('"',''),
    expected_output="Detailed execution plans suggesting how and when to execute trades for {stock_selection}.",
    agent=execution_agent,
)


In [11]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description="""
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    """.replace('"',''),
    expected_output="A comprehensive risk analysis report detailing potential risks and mitigation recommendations for {stock_selection}.",
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [12]:
# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model=model_name, temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [13]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [14]:
### this execution will take some time to run
ts_start = time()
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)
ts_stop = time()

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: 
        Continuously monitor and analyze market data for 
        the selected stock (AAPL). 
        Use statistical modeling and machine learning to 
        identify trends and predict market movements.
    


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: 
{
    "task": "Continuously monitor and analyze market data for AAPL",
    "context": "Use statistical modeling and machine learning to identify trends and predict market movements for AAPL",
    "co-worker": "Data Analyst"
}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to analyze trends and predict movements accurately.

Action:
Search the internet

Action Input:
{"search_query": "AAPL stock market data real-time"}
 


S

In [15]:
print(f"\n\nElapsed time: {(ts_stop-ts_start):.2f} sec")



Elapsed time: 64.30 sec


- Display the final result as Markdown.

In [16]:
Markdown(result)

Risk Analysis Report for AAPL:

Introduction:
Apple Inc. (AAPL) is a highly volatile stock with significant risk factors that traders need to consider when developing trading strategies and execution plans. In this report, we will analyze the potential risks associated with the approved trading strategies for AAPL and provide recommendations for mitigating these risks to optimize trading performance.

Risk Analysis:

1. Market Risk:
- AAPL is susceptible to market volatility, influenced by factors such as economic conditions, geopolitical events, and market sentiment. Traders must be prepared for sudden price fluctuations and market uncertainties.

Mitigation Strategy: Diversification of the trading portfolio to include other less volatile assets can help offset market risks associated with AAPL.

2. Stock-Specific Risk:
- AAPL's stock price can be impacted by company-specific events such as product launches, earnings reports, and regulatory changes. Traders need to monitor these events closely to anticipate potential price movements.

Mitigation Strategy: Implementing stop-loss orders and setting profit targets based on fundamental analysis can help manage stock-specific risks associated with AAPL.

3. Technical Risk:
- Technical indicators and analysis play a significant role in AAPL's price movements. Traders need to interpret technical signals accurately to make informed trading decisions.

Mitigation Strategy: Utilizing technical analysis tools such as moving averages, Fibonacci retracement levels, and chart patterns can help traders identify potential entry and exit points with lower technical risk.

4. Execution Risk:
- The successful execution of trading strategies for AAPL depends on timely and accurate order placement, trade monitoring, and risk management. Traders need to have efficient execution plans in place to minimize execution risk.

Mitigation Strategy: Utilizing automated trading systems and pre-defined criteria can help streamline trade execution processes and reduce the risk of manual errors.

Recommendations:
- Monitor market conditions and news updates regularly to stay informed about events that may impact AAPL's stock price.
- Implement risk management strategies such as stop-loss orders, profit targets, and diversification to protect trading capital and optimize returns.
- Continuously evaluate the performance of trading strategies and adjust them based on market trends and risk assessments.

Conclusion:
By conducting a thorough risk analysis of the proposed trading strategies for AAPL and implementing appropriate risk mitigation strategies, traders can navigate the volatile market conditions effectively and achieve their financial goals while minimizing potential losses. Stay informed, stay vigilant, and stay profitable when trading AAPL.

In [17]:
with open("financial_trading.md", "w") as f:
    f.write(result)